
This notebook ingests data from ~11K hungry-geese games. The purpose is to train a GBM model that, when given an Observation from the hungry-geese environment and a direction to move, predicts a Reward associated with a move in that direction.
Then, in real play, all 3 possible directions can be evaluated and the one with the highest expected Reward chosen.

Some assumptions: 
* there's _a_ function that can be estimated and then  used to generate a useful sequence of moves and that the data don't just encode various versions of lucky geese
* the "blurring" of positions I use in the data generation script is tuned to make inputs specific enough to act on for gain but general enough to so that the agent can generalize to future Observations (will test with a validation set, but there are on the order of 2^77 games, no set is quite big enough)
* The lightGBM model binaries are small enough and fast-enough to load to be used in the hungry geese environment

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, median_absolute_error
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
# data ingest
train_data = pd.read_csv("../input/naive-geese-data/train_data_1624151549.csv")
valid_data = pd.read_csv("../input/naive-geese-data/validation_data_1624128276.csv")

In [ ]:
# data prep
tX = train_data.drop(columns = ["Reward", "game", "step", "Unnamed: 0"])
ty = train_data["Reward"]
sX = valid_data.drop(columns = ["Reward", "game", "step", "Unnamed: 0"])
sy = valid_data["Reward"]


lg_data = lgb.Dataset(data = tX, label = ty)
valid_s = lg_data.create_valid(data=sX)

In [ ]:
np.array([1,4,5]).shape

In [ ]:
# core model training (TODO: cross-validate)
booster1 = lgb.train(train_set=lg_data, params={"n_estimators":500,
                                                "objective":"rmse",
                                                "boosting":"dart",
                                                "metric":"rmse", "learning_rate":0.0005},
 valid_sets=valid_s, verbose_eval=5,
 )


In [ ]:
# evaluation
py = booster1.predict(sX)

print(f"RMSE: {mean_squared_error(py,sy, squared=False)}")
# current best is 7.0
fig, ax = plt.subplots(1,1,figsize =(9,9))

plt.scatter(py, sy)
plt.show()

lgb.plot_importance(booster1)
plt.show()



In [ ]:
# check  that model was pickled 
with open("./lightgbm_geese.pkl", 'wb') as stream:
    pickle.dump(booster1.model_to_string(), stream)

with open("./lightgbm_geese.pkl", 'rb') as stream2:
    raw2  = pickle.load(stream2)

booster2 = lgb.Booster(model_str=raw2)
booster2.predict(sX)